In [ ]:
from cil.framework import AcquisitionGeometry, AcquisitionData

from cil.optimisation.algorithms import FISTA
from cil.optimisation.functions import ZeroFunction, LeastSquares

from cil.io import TIFFStackReader

from cil.processors import TransmissionAbsorptionConverter, Binner, Normaliser

from cil.plugins.astra import ProjectionOperator, FBP

from cil.plugins.ccpi_regularisation.functions import FGP_TV

from cil.utilities.display import show2D, show_geometry
from cil.utilities.jupyter import islicer, link_islicer

import numpy as np
import matplotlib.pyplot as plt
import os

# Lamniography reconstruction with TV regularisation using FISTA

This exercise uses CIL to reconstruct a Laminography dataset - that is a dataset with a tilted rotation axis. 

Laminography scanning is commonly used for large thin samples like circuit boards. In conventional CT these samples lead to widely varying path-lengths as the sample rotates. By tilting the sample and rotating it around a vector perpendicular to the sample plane the path-lengths stay relatively constant. However, this geometry leads to some artefacts in the reconstruction from the missing data. These artefacts are particularly prevelant when you look at a slice out of the sample plane.

By using regularisation, we can supress these artefacts. This notebook compares FDK, a least-squares with FISTA and Total-Variation with FISTA on a Lego phantom.

**Learning objectives:**

1. Construct an advanced AcquistionGeometry by-hand to describe the tilted and offset data

2. Use this geometry to read in a tiff stack and create an AcquisitionData object

3. Create a custom ImageGeometry around the thin-flat sample

4. Reconstruct the data with LS and TV

This example requires data from https://zenodo.org/record/2540509

https://zenodo.org/record/2540509/files/CLProjectionData.zip

https://zenodo.org/record/2540509/files/CLShadingCorrection.zip

Once downloaded update `path_common` to run the script.

In [ ]:
#Set the path to the directory containing the data
path_common = '/mnt/materials/SIRF/Fully3D/CIL/Laminography'

## Create the acquisition geometry

We know the system parameters from the paper and author clarification. We use this to set up a 3D cone-beam geometry with the rotation axis tilited 30&deg towards the source.


In [ ]:

#parameters are from the original paper/author clarification
src_to_det = 967.3209839
src_to_object = 295
tilt = 30. * np.pi / 180.
centre_of_rotation = 0.254 * 6.

mag = src_to_det / src_to_object 
object_offset_x = centre_of_rotation / mag

source_pos_y = -src_to_object
detector_pos_y = src_to_det-src_to_object
angles_list = -np.linspace(0, 360, 2513, endpoint=False)
num_pixels_x = 1596
num_pixels_y = 1148
pixel_size_xy = 0.254


ag = AcquisitionGeometry.create_Cone3D( source_position=[0.0, source_pos_y,0.0], \
                                        detector_position=[0.0, detector_pos_y,0.0],\
                                        rotation_axis_position=[object_offset_x,0,0],\
                                        rotation_axis_direction=[0,-np.sin(tilt), np.cos(tilt)] ) \
                        .set_angles(angles=angles_list, angle_unit='degree')\
                        .set_panel( num_pixels=[num_pixels_x, num_pixels_y], \
                                    pixel_size=pixel_size_xy,\
                                    origin='top-left')\
                        .set_labels(['angle','vertical','horizontal'])
print(ag)
show_geometry(ag)

# Read in the data
We will start by reading in data from a stack of tiffs. As we read in the data we will symmetrically crop and down-sample the data. We will remove a 100 pixel border from each projection, and we will only read in every 7 projections. This significantly reduces the computational time and memory cost without a loss of reconstruction quality.

As we are reading in a stack of tiffs we know that 'axis_0' is the angle, 'axis_1' is vertical and 'axis_2' is the horizontal. We want to crop in the horizontal and vertical directions, and slice the angles direction. We create a region of interest (RoI) dictionary with the axis name, and a tuple containing the starting pixel index, the end pixel index and the step size to slice.

We also need to update the geometry to account for the new panel size.



In [ ]:
crop = 100

roi = {'axis_0': (None, None, 7),
       'axis_1': (crop, -crop, None), 
       'axis_2': (crop, -crop, None)}

num_pixels_x = (1596 -2*crop)
num_pixels_y = (1148 -2*crop)
pixel_size_xy = 0.254

angles_list = -np.linspace(0, 360, int(2513/7), endpoint=False)

ag.set_angles(angles_list)
ag.set_panel(num_pixels=[num_pixels_x,num_pixels_y],pixel_size=pixel_size_xy,origin='top-left')
       

From `cil.io` we import and create a `TIFFStackReader` instance to read in the data, this is created with the directory path and the RoI dictionary defined above.

As we have already defined our acquisition geometry we can use the function `read_as_AcquisitionData()` to pass this to the reader. The reader will use this to configure and return an `AcquisitionData` object containing the data and the geometry describing it.

In [ ]:
path = 'Lego_Lamino30deg_XTH/'

reader = TIFFStackReader(file_name=os.path.join(path_common, path),roi=roi, mode = 'slice')
acq_data_raw = reader.read_as_AcquisitionData(ag)

islicer(acq_data_raw, direction='angle',origin='upper-left')


We now read in the dark and flat field images and use these to normalise the data. We need to crop these tiffs with the same RoI as previously. We now can use use `read()` here to simply read in the tiffs and return a numpy array which we delete after applying to the data.

In [ ]:
tiffs = [   os.path.join(path_common,'Lego_Lamino30deg_ShadingCorrection_XTH/Dark_80kV85uA.tif'),
            os.path.join(path_common,'Lego_Lamino30deg_ShadingCorrection_XTH/Flat_80kV85uA.tif') ]

roi = {'axis_0': (None, None, None),
       'axis_1': (crop, -crop, None), 
       'axis_2': (crop, -crop, None)}

reader = TIFFStackReader(file_name=tiffs, roi=roi)
dark_flat_data = reader.read()

normaliser = Normaliser(dark_flat_data[1], dark_flat_data[0])
acq_data_SC = normaliser(acq_data_raw)

islicer(acq_data_SC, direction='angle',origin='upper-left')

del acq_data_raw
del dark_flat_data

Finally we convert the intensity data to attenuation data using the Beer-Lambert law

In [ ]:

converter = TransmissionAbsorptionConverter()
acq_data_atten = converter(acq_data_SC)
islicer(acq_data_atten, direction='angle',origin='upper-left')

del acq_data_SC


We will run this notebook over 4x binned data for speed. The iterative reconstructions will take approximately 5 minutes at this binning.

We use CIL's Binner processor to average together every 4 pixels in the horizontal and vertical directions.

We define the RoI using the same syntax as previously, however now we set out start and stop indices to `None` as we want to include the full width of the data.

In [ ]:
#bin the data for speed 
binning = 4

roi = {'horizontal': (None, None, binning),
       'vertical': (None, None, binning)}
acq_data_binned = Binner(roi=roi)(acq_data_atten)

#note the number of pixels and pixel size is updated for you
print(acq_data_binned.geometry)

islicer(acq_data_binned,direction='angle',origin='upper-left')
del acq_data_atten

## Reconstructing the data using FDK

As we are using the ASTRA backend we need to reorder the data for use by ASTRA. If we were using TIGRE as the backend we could use `data.reorder('tigre')`

In [ ]:
acq_data = acq_data_binned
acq_data.reorder('astra')
ag = acq_data.geometry
print(ag.dimension_labels)

We use FDK to reconstruct the data. If we use the default `ImageGeomerty` then we will end up reconstructing a lot of empty space which becomes costly over many iterations. If we collapse the data along the Z and Y axes we can clearly identify a reconstruction window around the sample.

In [ ]:
ig_default = ag.get_ImageGeometry()
fbp = FBP(ig_default, ag)
FDK_reco = fbp(acq_data)
show2D([FDK_reco.max(axis=0),FDK_reco.max(axis=1)],title=['x-y plane','x-z plane'])

We can use the fast FDK reconstruction to modify this reconstruction window to remove as much of the empty space as possible, whilst keeping every voxel that will contain the object. Below we shrink and offset the window, but keep the voxel size as the pixel size scaled by magnification.

In [ ]:

ig = ag.get_ImageGeometry()
ig.voxel_num_z=130
ig.voxel_num_y=240
ig.voxel_num_x=310

ig.center_x=15*ig.voxel_size_x
ig.center_z=-10*ig.voxel_size_z

fbp = FBP(ig, ag)
FDK_reco = fbp(acq_data)

show2D([FDK_reco.max(axis=0),FDK_reco.max(axis=1)],title=['x-y plane','x-z plane'])
show_geometry(ag,ig)


Some slices from the FDK reconstruction are shown below. Along the `vertical` direction (x-y plane) we can see some ghosting of the object in the nearest layer. If we slice instead along the `horizontal_x` and `horizontal_y` directions we can clearly see missing data wedge artefacts which cause this this ghosting. This is usual for laminography datasets and the reconstructions are often analysed only in 2D slices.

In [ ]:
slice_list=[('vertical',76),('vertical',55),('horizontal_x',155),('horizontal_y',138)]
show2D(FDK_reco,slice_list=slice_list, title="FDK reconstruction", fix_range=(-0.02,0.07))

## Reconstructing the data using Least Squares with FISTA

Using our ImageGeometry (ig) and AcquisitionGeometry (ag) we define our projector and a data-fidelity LeastSquares term.

We can use FISTA to iteratively solve this reconstruction. As there is no regularisation term we will stop at 100 iterations and observe a similar reconstruction to that obtained by FDK. This example will take approximatly 1 minute to run.


In [ ]:
Projector = ProjectionOperator(ig, ag)
LS = LeastSquares(A=Projector, b=acq_data)
fista_LS = FISTA(initial=FDK_reco, f=LS, g=ZeroFunction(), max_iteration=1000, update_objective_interval=10)


In [ ]:
fista_LS.run(100)
LS_reco = fista_LS.get_output()
show2D(LS_reco,slice_list=slice_list, title="LS reconstruction", fix_range=(-0.02,0.07))


## Reconstructing the data using Total Variation regularised Least Squares with FISTA

We reuse the LeastSquares function, but now we also can define a Total-Variation function. In this example we use FGP_TV from the CCPi-RegularisationToolkit with the `gpu` backend.

We also can add a non-negativity constraint to the function.

Again, we set up and use FISTA to iteratively solve this reconstruction. We run this until the background appears uniform suggesting TV has converged.

For this example we will run 100 iterations, which will take approximately 3 minutes to run.

In [ ]:
alpha = 0.05
TV = FGP_TV(alpha=alpha, nonnegativity=True, device='gpu',)
fista_TV = FISTA(initial=FDK_reco, f=LS, g=TV, max_iteration=1000, update_objective_interval=10)

In [ ]:
fista_TV.run(100)
TV_reco = fista_TV.get_output()
show2D(TV_reco,slice_list=slice_list, title="TV reconstruction", fix_range=(-0.02,0.07))

## Comparing the results

We can compare the results of the three reconstructions. Using Total-Variation we have supressed the missing data artefacts and the ghosting, which would allow us to perform a cleaner segmentation of this sample.

In [ ]:
for slice in slice_list:
    show2D([FDK_reco,LS_reco,TV_reco],slice_list=slice, title=['FDK','LS','TV'], fix_range=(-0.02,0.07),num_cols=3)